In [70]:
import pandas as pd
import datetime as dt
import numpy as np

## Loading Data

In [76]:
df1 = pd.read_csv(r"C:\Users\mohap\OneDrive\Desktop\IIMN\TERM-IV\BDMDM\Assignment 1\online_retail_II_2009_10.csv", encoding='unicode_escape')
df2 = pd.read_csv(r"C:\Users\mohap\OneDrive\Desktop\IIMN\TERM-IV\BDMDM\Assignment 1\online_retail_II_2010_11.csv", encoding='unicode_escape')

## Concatinanting both datframes in Row-wise mode

In [77]:
df=pd.concat([df2,df1],axis=0)
df.shape

(1067371, 8)

## Resetting Index

In [78]:
df.reset_index(drop=True,inplace=True)
df["Invoice"][0]

'536365'

In [79]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [19]:
import sweetviz as sv
report=sv.analyze(df[['Quantity', 'InvoiceDate','Price', 'Customer ID', 'Country']])
report.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## Removing records with Negative values of price, Quantity
## Further, removing records with missing values of Invoice, InvoiceDate, Description

In [80]:
df=df.loc[df["Quantity"]>0,]
df=df.loc[df["Price"]>0,]
df.dropna(subset=['InvoiceDate', 'Invoice',"Description"],inplace=True)

df.shape

(1041671, 8)

In [81]:
df.describe(include="all")

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
count,1041671,1041671,1041671,1.041671e+06,1041671,1.041671e+06,805549.000000,1041671
unique,40078,4917,5399,NaN,37213,NaN,NaN,43
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,06-12-2010 16:57,NaN,NaN,United Kingdom
freq,1350,5686,5778,NaN,1350,NaN,NaN,958502
mean,NaN,NaN,NaN,1.096345e+01,NaN,4.077038e+00,15331.954970,NaN
std,NaN,NaN,NaN,1.265149e+02,NaN,5.144898e+01,1696.737039,NaN
min,NaN,NaN,NaN,1.000000e+00,NaN,1.000000e-03,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000e+00,NaN,1.250000e+00,13982.000000,NaN
50%,NaN,NaN,NaN,3.000000e+00,NaN,2.100000e+00,15271.000000,NaN
75%,NaN,NaN,NaN,1.000000e+01,NaN,4.130000e+00,16805.000000,NaN


In [82]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [83]:
from datetime import datetime, date

In [84]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True)
df["Customer ID"]=df["Customer ID"].astype('str')

##Adding two new columns by extracting year and month from date

df["Year"]=[x.year for x in df["InvoiceDate"]]
df["Month"]=[x.month for x in df["InvoiceDate"]]

### Adding new column with revenue value

df["Revenue"]=df["Quantity"]*df["Price"]

In [85]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID            object
Country                object
Year                    int64
Month                   int64
Revenue               float64
dtype: object

In [86]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,20.34


## Compute number of unique months for each product during which it is sold

In [87]:
report=df.groupby(["Description","Year"]).agg(
{
    'Month':"nunique"
}
)

In [88]:
report.reset_index()

,Description,Year,Month
0,DOORMAT UNION JACK GUNS AND ROSES,2010,3
1,3 STRIPEY MICE FELTCRAFT,2009,1
2,3 STRIPEY MICE FELTCRAFT,2010,7
3,4 PURPLE FLOCK DINNER CANDLES,2010,5
4,4 PURPLE FLOCK DINNER CANDLES,2011,11
...,...,...,...
11336,ZINC WILLIE WINKIE CANDLE STICK,2009,1
11337,ZINC WILLIE WINKIE CANDLE STICK,2010,12
11338,ZINC WILLIE WINKIE CANDLE STICK,2011,12
11339,ZINC WIRE KITCHEN ORGANISER,2011,5


## Sum function on month to get number of months during which product had non zero sale during both the years

In [89]:
counts_report=report.groupby(["Description"]).agg({
    "Month":sum
})

In [90]:
counts_report=counts_report.reset_index()

## Select only those products that had non-zero sale for atleast 24 months during both years. generate a list of such Product descriptons that qualify this criterion

In [91]:
counts_report=counts_report.loc[counts_report["Month"]>=24,]
counts_report

,Description,Month
45,12 EGG HOUSE PAINTED WOOD,25
54,12 PENCILS SMALL TUBE SKULL,25
58,12 PENCILS TALL TUBE SKULLS,25
59,12 PENCILS TALL TUBE WOODLAND,25
62,12 RED ROSE PEG PLACE SETTINGS,25
...,...,...
5325,WRAP PINK FAIRY CAKES,25
5367,YOU'RE CONFUSING ME METAL SIGN,24
5382,ZINC HEART LATTICE T-LIGHT HOLDER,25
5386,ZINC METAL HEART DECORATION,25


In [92]:
products=list(counts_report["Description"])
products

['12 EGG HOUSE PAINTED WOOD',
 '12 PENCILS SMALL TUBE SKULL',
 '12 PENCILS TALL TUBE SKULLS',
 '12 PENCILS TALL TUBE WOODLAND',
 '12 RED ROSE PEG PLACE SETTINGS',
 '200 BENDY SKULL STRAWS',
 '200 RED + WHITE BENDY STRAWS',
 '3 HOOK PHOTO SHELF ANTIQUE WHITE',
 '36 PENCILS TUBE SKULLS',
 '36 PENCILS TUBE WOODLAND',
 '3D DOG PICTURE PLAYING CARDS',
 '3D SHEET OF CAT STICKERS',
 '3D SHEET OF DOG STICKERS',
 '4 IVORY DINNER CANDLES SILVER FLOCK',
 '4 LAVENDER BOTANICAL DINNER CANDLES',
 '4 PEAR BOTANICAL DINNER CANDLES',
 '4 VANILLA BOTANICAL CANDLES',
 '6 CHOCOLATE LOVE HEART T-LIGHTS',
 '6 EGG HOUSE PAINTED WOOD',
 '6 RIBBONS ELEGANT CHRISTMAS ',
 '6 RIBBONS EMPIRE  ',
 '6 RIBBONS RUSTIC CHARM',
 '6 RIBBONS SHIMMERING PINKS ',
 '60 TEATIME FAIRY CAKE CASES',
 '72 SWEETHEART FAIRY CAKE CASES',
 '75 GREEN FAIRY CAKE CASES',
 '75 GREEN PETIT FOUR CASES',
 'ABSTRACT CIRCLE JOURNAL ',
 'ABSTRACT CIRCLES POCKET BOOK',
 'ABSTRACT CIRCLES SKETCHBOOK ',
 'AGED GLASS SILVER T-LIGHT HOLDER',
 'AIRL

## Var function on group by using prod desc, year, month to get variance of only those products which appeard in the list generted in last step

In [93]:
Monthly_Sales=df.loc[df["Description"].isin(products),].groupby(["Description","Year","Month"]).agg(
{
    "Quantity":sum
})

In [94]:
Monthly_Sales=Monthly_Sales.reset_index()

In [95]:
Monthly_Sales=Monthly_Sales.groupby(["Description"]).agg({
    "Quantity":"var"
})

In [96]:
Monthly_Sales=Monthly_Sales.reset_index()
Monthly_Sales

,Description,Quantity
0,12 EGG HOUSE PAINTED WOOD,1174.250000
1,12 PENCILS SMALL TUBE SKULL,92326.173333
2,12 PENCILS TALL TUBE SKULLS,23696.360000
3,12 PENCILS TALL TUBE WOODLAND,12375.543333
4,12 RED ROSE PEG PLACE SETTINGS,1239.493333
...,...,...
735,WRAP PINK FAIRY CAKES,129496.923333
736,YOU'RE CONFUSING ME METAL SIGN,65990.259058
737,ZINC HEART LATTICE T-LIGHT HOLDER,1520.223333
738,ZINC METAL HEART DECORATION,224021.456667


## Sort the resulting dataframe in descending order and select top 5 products with the least variance and most stable sales 

In [97]:
Monthly_Sales.sort_values(by=["Quantity"]).head(5)

,Description,Quantity
400,OLD ROSE COMBO BEAD NECKLACE,13.346014
690,VINTAGE SHELLS PRINT,18.282609
524,ROSE DU SUD WASHBAG,21.456667
688,VINTAGE ROSE BEAD BRACELET RASPBERR,21.731884
626,SQUARE FLOOR CUSHION VINTAGE RED,27.152174


# Generating RFM Labels for Customers

In [98]:
RFM=df.groupby("Customer ID").agg({
    "Invoice":"nunique",
    "Revenue":"sum",
    "InvoiceDate": max
})

In [99]:
RFM=RFM.reset_index()
RFM

,Customer ID,Invoice,Revenue,InvoiceDate
0,12346.0,12,77556.46,2011-01-18 10:01:00
1,12347.0,8,5633.32,2011-12-07 15:52:00
2,12348.0,5,2019.40,2011-09-25 13:13:00
3,12349.0,4,4428.69,2011-11-21 09:51:00
4,12350.0,1,334.40,2011-02-02 16:01:00
...,...,...,...,...
5874,18284.0,1,461.68,2010-10-04 11:33:00
5875,18285.0,1,427.00,2010-02-17 10:24:00
5876,18286.0,2,1296.43,2010-08-20 11:57:00
5877,18287.0,7,4182.99,2011-10-28 09:29:00


In [100]:
max_date=df["InvoiceDate"].max()
max_date

Timestamp('2011-12-09 12:50:00')

## Generating recency score for customers using difference between latest transaction date for customer and maximum date avaibale in dataset

In [101]:
RFM["Recency"]=[(max_date-date).days for date in RFM["InvoiceDate"]]
RFM

,Customer ID,Invoice,Revenue,InvoiceDate,Recency
0,12346.0,12,77556.46,2011-01-18 10:01:00,325
1,12347.0,8,5633.32,2011-12-07 15:52:00,1
2,12348.0,5,2019.40,2011-09-25 13:13:00,74
3,12349.0,4,4428.69,2011-11-21 09:51:00,18
4,12350.0,1,334.40,2011-02-02 16:01:00,309
...,...,...,...,...,...
5874,18284.0,1,461.68,2010-10-04 11:33:00,431
5875,18285.0,1,427.00,2010-02-17 10:24:00,660
5876,18286.0,2,1296.43,2010-08-20 11:57:00,476
5877,18287.0,7,4182.99,2011-10-28 09:29:00,42


In [102]:
RFM.columns

Index(['Customer ID', 'Invoice', 'Revenue', 'InvoiceDate', 'Recency'], dtype='object')

In [104]:
### Rearranging column sequence 
RFM=RFM[['Customer ID', 'Recency','Invoice', 'Revenue' ]]

### Renaming the columns as RFM
RFM.columns=["Customer ID",'Recency',"Frequency","Monetary"]

### Retainign only those values which are not nan 
RFM=RFM.loc[RFM["Customer ID"]!='nan',]

### Generating ranking of Recency score
RFM["Recency_Label"]=pd.qcut(RFM["Recency"],5,labels=[5,4,3,2,1])

### Generating ranking of frequency score
RFM["Frequency_Label"]=pd.qcut(RFM["Frequency"],q=[0, .30, 0.50,.70,0.90, 1.] ,labels=[1,2,3,4,5])

### generating ranking of Monetary Score
RFM["Monetary_Label"]=pd.qcut(RFM["Monetary"],5 ,labels=[1,2,3,4,5])

RFM

,Customer ID,Recency,Frequency,Monetary,Recency_Label,Frequency_Label,Monetary_Label
0,12346.0,325,12,77556.46,2,4,5
1,12347.0,1,8,5633.32,5,4,5
2,12348.0,74,5,2019.40,3,3,4
3,12349.0,18,4,4428.69,5,3,5
4,12350.0,309,1,334.40,2,1,2
...,...,...,...,...,...,...,...
5873,18283.0,3,22,2736.65,5,5,4
5874,18284.0,431,1,461.68,1,1,2
5875,18285.0,660,1,427.00,1,1,2
5876,18286.0,476,2,1296.43,1,1,4


In [63]:
RFM.dtypes

Customer ID          object
Recency               int64
Frequency             int64
Monetary            float64
Recencey_Label     category
Frequency_Label    category
Monetary_Label     category
dtype: object

In [105]:
def assign_segment(row):
    if row['Recency_Label'] >= 4 and row['Frequency_Label'] >= 4 and row['Monetary_Label'] >= 4:
        return 'Champions'
    elif row['Recency_Label'] >= 3 and row['Frequency_Label'] >= 3:
        return 'Loyal Customers'
    elif row['Recency_Label'] >= 4 and row['Frequency_Label'] >= 3:
        return 'Potential Loyalists'
    elif row['Recency_Label'] >= 4:
        return 'New Customers'
    elif row['Recency_Label'] >= 3 and row['Frequency_Label'] >= 2 and row['Monetary_Label'] >= 2:
        return 'Promising'
    elif row['Recency_Label'] >= 2 and row['Frequency_Label'] < 3:
        return 'Needs Attention'
    elif row['Recency_Label'] < 2 and row['Frequency_Label'] >= 2:
        return 'About to Sleep'
    elif row['Recency_Label'] < 2 and row['Frequency_Label'] < 2:
        return 'At Risk'
    elif row['Recency_Label'] < 3 and row['Monetary_Label'] >= 3:
        return 'Can\'t Lose Them'
    elif row['Recency_Label'] < 3 and row['Frequency_Label'] < 3 and row['Monetary_Label'] < 3:
        return 'Hibernating'
    else:
        return 'Lost'

# Apply the segmentation function to each row
RFM['Segment'] = RFM.apply(assign_segment, axis=1)

# Display the updated DataFrame
RFM.head()

,Customer ID,Recency,Frequency,Monetary,Recency_Label,Frequency_Label,Monetary_Label,Segment
0,12346.0,325,12,77556.46,2,4,5,Can't Lose Them
1,12347.0,1,8,5633.32,5,4,5,Champions
2,12348.0,74,5,2019.40,3,3,4,Loyal Customers
3,12349.0,18,4,4428.69,5,3,5,Loyal Customers
4,12350.0,309,1,334.40,2,1,2,Needs Attention


In [108]:
from IPython.display import Image 
  
# get the image 
Image(url="segment.png") 